# LANE DRIVING
María Rodríguez Palomo

In [1]:
import glob
import os
import sys
import carla
import argparse
import random
import time
import math
import numpy as np
import cv2

#### VEHICLE CREATION

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
client.load_world("Town05")
world = client.get_world()
m = world.get_map()

In [3]:
loc2=carla.Transform(carla.Location(x=-174.557922, y=185.525208, z=13.138501), carla.Rotation(pitch=0.733207, yaw=-147.194565, roll=0.006472))
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')

vehicle = world.spawn_actor(bp, loc2)

#### VIEW CHANGE

In [4]:
spectator = world.get_spectator()
spectator_pos = carla.Transform(loc2.location + carla.Location(x=10,y=10,z=4),carla.Rotation(yaw=loc2.rotation.yaw))
spectator.set_transform(spectator_pos)

#### CONSTANT SPEED

In [5]:
preferred_speed = 90
speed_threshold = 2

def maintain_speed(s):
    #The return is the acceleration
    if s>=preferred_speed:
        return 0
    elif s < preferred_speed - speed_threshold:
        return 0.8
    else:
        return 0.4

#### ANGLE

In [6]:
#function to subtract 2 vectors
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))
    return degrees

#### DEVELOPER VIEW

In [7]:
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

In [8]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5
camera_bp=world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')
camera_init_trans= carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera=world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
image_w= camera_bp.get_attribute('image_size_x').as_int()
image_h= camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

#### DRIVING

In [9]:
font=cv2.FONT_HERSHEY_SIMPLEX
org2=(30,50)
fontScale=0.5
color=(255,255,255)
thickness=1
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
quit=False

while True:
    #Camera
    world.tick()
    if cv2.waitKey(1)==ord('q'): #If we press the q key, the tab disappears
        quit=True
        break
    image = camera_data['image']
    waypoint_separation = 5
    loc=vehicle.get_location()
    current_w = m.get_waypoint(loc)
    map = world.get_map()
    next_w0 = list(current_w.next(waypoint_separation))[0]
    next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    x=vehicle.get_transform().location.x
    y=vehicle.get_transform().location.y
    MAX_STEER_DEGREES = 7
    velocity = vehicle.get_velocity()
    speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
    image=cv2.putText(image,'Speed: '+str(int(speed))+' km/h', org2,font,fontScale,color,thickness,cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    predicted_angle = get_angle(vehicle,next_w)
    
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-180:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 180:
        predicted_angle = predicted_angle-360
    steer_input = predicted_angle
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    steer_input = steer_input/75
    if steer_input > 0.05:
        estimated_throttle=0
    vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steer_input))
    cv2.imshow('RGB Camera', image)
cv2.destroyAllWindows()
camera.stop()

#### DESTROY

In [10]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()